In [39]:
# 載入套件
import numpy as np
import cv2 as cv

# 讀取影像
# img = cv.imread('ball.jpg')
# img = cv.imread('light_ball.jpg')
img = cv.imread('contrasthigh.jpg')
# img = cv.imread('doraemon.jpg')
img = cv.resize(img, (400, 300))
# 轉換色彩空間 (RGB 彩色圖片由 OpenCV 讀取後，通道順序為 BGR)
img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

In [34]:
# 設定顏色區間上下界
initial_lower = np.array([100, 0, 0])
initial_upper = np.array([124, 255, 255])
H_tolerance = 30
S_tolerance = 160
V_tolerance = 160
max_iterations = 1000

In [40]:
def count_objects(lower, upper):
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    mask = cv.inRange(hsv, lower, upper)
    result = cv.bitwise_and(img, img, mask=mask)
    return cv.countNonZero(mask)

lower = np.array([100, 0, 0])
upper = np.array([124, 255, 255])

while True:
    middle = (lower + upper) // 2
    object_count = count_objects(lower, middle)
    
    # 調整搜尋範圍
    if object_count < 20:
        lower = middle
    else:
        upper = middle
    
    
    if np.array_equal(lower, upper) or np.array_equal(lower + 1, upper):
        break
print(lower,upper)
# 最終的HSV上下限值
hsv_lower = upper - np.array([H_tolerance , S_tolerance, V_tolerance])
hsv_upper = upper + np.array([H_tolerance , S_tolerance, V_tolerance])

hsv_lower[0] = max(0, min(hsv_lower[0], 180))
hsv_upper[0] = max(0, min(hsv_upper[0], 180))
hsv_lower[1:] = np.clip(hsv_lower[1:], 0, 255)
hsv_upper[1:] = np.clip(hsv_upper[1:], 0, 255)

print("最終的HSV上限值：", hsv_upper)
print("最終的HSV下限值：", hsv_lower)


[111 126 126] [112 127 127]
最終的HSV上限值： [142 255 255]
最終的HSV下限值： [82  0  0]


In [36]:
# 透過 inRange() 函式形成白色遮罩，遮罩區域為原始影像藍色區域
hsv_bin_mask = cv.inRange(img_hsv, hsv_lower, hsv_upper)

# 顯示影像
cv.imshow("HSV_mask",hsv_bin_mask)

# 按下任意鍵關閉視窗
cv.waitKey(0)
cv.destroyAllWindows()


# 套用 hsv_bin_mask 到原圖上進行過濾
hsv_mask = cv.bitwise_and(img, img, None, mask=hsv_bin_mask)

# 顯示過濾後的影像
cv.imshow("HSV_mask_result", hsv_mask)

# 按下任意鍵關閉視窗
cv.waitKey(0)
cv.destroyAllWindows()

In [41]:
# 將我們的 HSV Mask 套用OpenCV內建的Contours輪廓函式，在我們目標物體畫出邊框
# 找輪廓
(cnts, _) = cv.findContours(hsv_bin_mask.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)  # 回傳輪廓list

copy_img_1=img.copy()
copy_img_2 = img.copy()
clone= img.copy()

cv.drawContours(clone, cnts, -1, (0, 255, 0), 2)

# for 遍歷 cnts 裡的每一個輪廓
for cnt in cnts:
    (x, y, w, h) = cv.boundingRect(cnt)  # 矩形 rect
    print('x,y,w,h:',x,y,w,h)
    cv.rectangle(copy_img_1, (x, y), (x + w, y + h), (0, 0, 255), 1)  # 畫矩形
    
    BoundingBox = cv.minAreaRect(cnt)  # 最小外接矩形的中心（x，y），（寬度，高度），旋轉角度）
    BoundingBox = np.int0(cv.boxPoints(BoundingBox))  # int0 會省略小數點後方的數字
    cv.drawContours(copy_img_2, [BoundingBox], -1, (0, 255, 0), 1)

# 顯示影像
cv.imshow('Contour', clone)  # 輪廓
cv.imshow('Bounding Rectangle', copy_img_1)  # 矩形
cv.imshow('Minimum Area Rectangle', copy_img_2)  # 最小外接矩形

# 按下任意鍵關閉視窗
cv.waitKey(0)
cv.destroyAllWindows()


x,y,w,h: 200 250 55 28
x,y,w,h: 37 235 1 1
x,y,w,h: 82 232 1 1
x,y,w,h: 38 220 47 25
x,y,w,h: 173 159 1 1
x,y,w,h: 343 153 2 2
x,y,w,h: 165 152 3 2
x,y,w,h: 347 151 2 1
x,y,w,h: 325 145 3 2
x,y,w,h: 334 144 1 1
x,y,w,h: 323 143 1 2
x,y,w,h: 158 143 1 1
x,y,w,h: 320 142 1 1
x,y,w,h: 308 136 1 1
x,y,w,h: 306 134 1 1
x,y,w,h: 315 133 2 2
x,y,w,h: 313 132 1 1
x,y,w,h: 302 132 1 1
x,y,w,h: 294 128 1 1
x,y,w,h: 290 126 1 1
x,y,w,h: 300 125 1 1
x,y,w,h: 288 125 1 1
x,y,w,h: 149 125 89 98
x,y,w,h: 321 124 1 1
x,y,w,h: 315 124 4 3
x,y,w,h: 310 123 2 2
x,y,w,h: 285 123 1 1
x,y,w,h: 304 121 1 1
x,y,w,h: 281 121 1 1
x,y,w,h: 279 120 1 1
x,y,w,h: 302 117 10 5
x,y,w,h: 300 117 1 1
x,y,w,h: 272 116 1 1
x,y,w,h: 269 114 1 1
x,y,w,h: 314 112 1 1
x,y,w,h: 291 112 1 1
x,y,w,h: 277 112 3 1
x,y,w,h: 265 112 1 1
x,y,w,h: 268 108 8 4
x,y,w,h: 289 107 17 10
x,y,w,h: 276 107 1 1
x,y,w,h: 297 106 3 2
x,y,w,h: 273 106 2 2
x,y,w,h: 266 106 1 1
x,y,w,h: 254 106 9 5
x,y,w,h: 294 105 1 1
x,y,w,h: 284 104 1 1
x,y,w,h